## EJERCICIO 2: MODELO DE SIMULACIÓN. EL MODELO.

#### CARGA DE BIBLIOTECAS

In [1]:
import sqlite3      
import matplotlib.pyplot as plt
import fitter as ft
import simpy as sim
import numpy as np
import datetime as dt
import pandas as pd
from scipy.stats import f
from math import ceil

## **DATOS**

In [2]:
worked_minutes = 8*60

# CONSULTAS A LA BASE DE DATOS

conexion=sqlite3.connect("/Users/lucianosmith/Documents/Investigación Operativa 3/2C2024/Python/C07/integrador.db")

# PLAN DE PRODUCCIÓN

# fecha de inicio de la planificacion y final de planificación

m = conexion.execute(f"select min(sch_startdate) from schedule;")

start_date = dt.datetime.strptime(m.fetchall()[0][0], '%Y-%m-%d %H:%M:%S')

m = conexion.execute(f"select max(sch_enddate) from schedule;")

end_date = dt.datetime.strptime(m.fetchall()[0][0], '%Y-%m-%d %H:%M:%S')

# SECUENCIAS

# secuencia incial (lote, producto, cantidad, siguiente_wc, color)

start_seq = {}  # secuencia inicial
seq = {}        # secuencia completa

m = conexion.execute(f"select sch_task, sch_lot, sch_product, sal_qty, (select p.prs_workcenter from process p where p.prs_product = sch_product and p.prs_prev=x.prs_code), sch_color from schedule, process x, sales where sal_so=sch_lot and x.prs_workcenter=sch_task and x.prs_product=sch_product and x.prs_prev=-1 order by sch_startdate asc;")

for i in m.fetchall():
    if i[0] in start_seq: 
        start_seq[i[0]].append((i[1], i[2], i[3], i[4], i[5])) # si existe el worck center hace un append
        seq[i[0]].append((i[1], i[2], i[3], i[4], i[5]))
    else:
        start_seq.update({i[0]: [(i[1], i[2], i[3], i[4], i[5])]})
        seq.update({i[0]: [(i[1], i[2], i[3], i[4], i[5])]})

# secuencia completa

m = conexion.execute(f"select sch_task, sch_lot, sch_product, sal_qty, (select p.prs_workcenter from process p where p.prs_product = sch_product and p.prs_prev=x.prs_code), sch_color from schedule, process x, sales where sal_so=sch_lot and x.prs_workcenter=sch_task and x.prs_product=sch_product and x.prs_prev>-1 order by sch_startdate asc;")

for i in m.fetchall():
    if i[0] in seq: 
        seq[i[0]].append((i[1], i[2], i[3], i[4], i[5])) 
    else:
        seq.update({i[0]: [(i[1], i[2], i[3], i[4], i[5])]})

# DATOS DE LOS PROCESOS

# areas y cantidad de empleados

wc = {}
wcq = conexion.execute(f"select woc_code, woc_employees from work_center;")
for i in wcq:
    wc.update({i[0]: i[1]})

# tiempos de los procesos (por producto) 

m = conexion.execute(f"select prs_product, prs_workcenter, prs_time, prs_deviation, prs_distribution from process;")

t_process = {}

for i in m.fetchall(): 
        t_process.update({(i[0], i[1]): (i[4], i[2], i[3])})

# consumo de materia prima (por proceso y producto)

m = conexion.execute(f"select prs_product, prs_workcenter, prs_rawqty from process;")

t_rawqty = {}

for i in m.fetchall(): 
        t_rawqty.update({(i[0], i[1]): i[2]})

# matriz de cambio de color (pintura)

m = conexion.execute(f"select c1.col_desc, c2.col_desc, cot_time from color_table, colors c1, colors c2 where c1.col_code = cot_orig and c2.col_code = cot_dest;")

setup_color_matrix = {}

for i in m.fetchall(): 
        setup_color_matrix.update({(i[0], i[1]): (i[2])})     

# ausentismo para el area de CORTE

qa = conexion.execute(f"select count(*) from attendance where att_attendance = 0 and att_workcenter = 'CORTE';")
qt = conexion.execute(f"select count(*) from attendance where att_workcenter = 'CORTE';")

p_ausentismo = qa.fetchall()[0][0]/qt.fetchall()[0][0]

# feriados

s = conexion.execute(f"select * from feriados;")
holidays = []
for i in s.fetchall():
    holidays.append( (i[0],i[1],i[2]) )

# INFORMACION DE ALMACENAMIENTO

# stock de materia prima

s = conexion.execute(f"select * from stock;")
stk = {}
for i in s.fetchall():
    stk.update({i[1]: i[3]})

# reaprovisionamiento

def get_min(fecha):
    diff = fecha - start_date
    min = 0
    day = start_date
    for d in range(diff.days):        
        if (day.weekday()<5) and ((day.day, day.month, day.year) not in holidays): 
            min += worked_minutes
        day = day + dt.timedelta(days=1)
    return min

reaprovisionamiento = []

s = conexion.execute(f"select * from to_receive where tre_status='waiting' order by tre_date asc;")

t0 = 0
for i in s.fetchall():
    tt = get_min(dt.datetime.strptime(i[4], '%Y-%m-%d'))
    reaprovisionamiento.append((tt-t0, i[2], i[3]))
    t0 = tt

# AJUSTE DE TIEMPOS DE MANTENIMIENTO

# DOBLADORA (TTR)

m = conexion.execute(f"select mai_ttr from maintenance where mai_machine='DOBLADORA' and mai_type='C';")
d_ttr = [i[0] for i in m.fetchall()] 

# ajuste y obtención de parámetros

f2 = ft.Fitter(d_ttr, bins=20, timeout=100, distributions=['norm', 'expon', 'weibull_max','weibull_min'])
f2.fit()

u_ttr = f2.get_best()['norm'][0]
d_ttr = f2.get_best()['norm'][1]

# DOBLADORA (TBF)

m = conexion.execute(f"select mai_tbf from maintenance where mai_machine='DOBLADORA' and mai_type='C';")
d_tbf = [i[0] for i in m.fetchall()] 

# ajuste y obtención de parámetros
f1 = ft.Fitter(d_tbf, bins=20, timeout=100, distributions=['norm', 'expon','beta', 'f'])
f1.fit()

a_tbf = f1.get_best()['f'][0]
b_tbf = f1.get_best()['f'][1]
c_tbf = f1.get_best()['f'][2]
d_tbf = f1.get_best()['f'][3]


## **MODELO DE SIMULACIÓN**

In [3]:
class modelo1:

    def __init__(self, start_date, start_seq, seq, wc, t_process, t_rawqty, color_matrix, p_ausentismo, holidays, stk, reaprov, u_ttr, d_ttr, a_tbf, b_tbf, c_tbf, d_tbf, tmax):

        self.env = sim.Environment()
        
        # PARÁMETROS INTERNOS DE CONFIGURACIÓN
        
        self.worked_minutes = 8*60                      # minutos trabajados por día         
        self.setup_color = 'rojo'                       # estado de configuración de la zona de pintura
        self.last_maintenance = 900                     # hace cuanto se realizó el último mantenimiento de la dobladora (min)
        self.continuar = True                           # flag para parar la simulación
        self.mant = False                               # flag que indica si la dobladora se encuentra en mantenimiento

        # PARÁMETROS RECIBIDOS
        self.start_date = start_date                    # fecha de inicio planificada
        self.start_seq = start_seq                      # secuencia de arranque (para la carga de los almacenes virtuales de inicio)
        self.seq = seq                                  # secuencia completa producto de la planificación
        self.wc = wc                                    # información de los centros de trabajo
        self.t_process = t_process                      # tiempos de proceso (según producto y workcenter)
        self.t_rawqty = t_rawqty                        # información de los consumos de materia prima por producto y workcenter
        self.setup_color_matrix = color_matrix          # matriz de cambio de color
        self.p_ausentismo = p_ausentismo                # probabilidad de ausentismo
        self.holidays = holidays                        # información sobre los feriados
        self.stk = stk                                  # información de stock
        self.reaprov = reaprov.copy()                   # información de los pedidos de materia prima que se van a recibir
        self.u_ttr = u_ttr                              # variables con los parámetros de las distribuciones de manentenimiento (TTR)
        self.d_ttr = d_ttr                              # variables con los parámetros de las distribuciones de manentenimiento (TTR)
        self.a_tbf = a_tbf                              # variables con los parámetros de las distribuciones de manentenimiento (TBF)
        self.b_tbf = b_tbf                              # variables con los parámetros de las distribuciones de manentenimiento (TBF)
        self.c_tbf = c_tbf                              # variables con los parámetros de las distribuciones de manentenimiento (TBF)
        self.d_tbf = d_tbf                              # variables con los parámetros de las distribuciones de manentenimiento (TBF)
        self.tmax = tmax

        # CARGA DE OBJETOS DEL MODELO
        
        self.wc.update({'terminado':1})                 # agrega la operación final para el almacenamiento de productos terminados

        self.store = {}                                 # define los almacenes de semielaborados y terminados
        for i in list(self.wc.keys()):
            self.store.update({i:sim.FilterStore(self.env)})

        self.raw_material = {}                          # define los almacenes de materia prima
        for i in self.stk:
            self.raw_material.update({i: sim.Container(self.env, init=self.stk[i])})

        self.workers = {}                               # define la disponibilidad de recursos
        for i in list(self.wc.keys()):                  # los operarios de corte_m los maneja como contenedores
            if i!='corte_m':                        
                self.workers.update({i: sim.Resource(self.env, capacity=self.wc[i])})
            else:                                   
                self.workers_corte_m = sim.Container(self.env, capacity=self.wc[i], init=self.wc[i])

        for i in self.start_seq:                        # carga el almacén inicial para cada area de trabajo                                                       
            for j in self.start_seq[i]:
                self.store[i].put([j[0], j[1]])

    # --------------------------------------------------------------------------------------------------------

    # FUNCIONES AUXILIARES

    # devuelve el tiempo de setup de cambio de color 
    # en pintura teniendo en cuenta color actual y
    # próximo color.

    def get_setupTime(self, next_color):                
        self.setup_color
        t = self.setup_color_matrix[(self.setup_color, next_color)]
        self.setup_color = next_color
        return t
    
    # devuelve el tiempo de proceso para un producto, 
    # cantidad y centro de trabajo, la distribución 
    # se obtiene de la base de datos.

    def get_processTime(self, prod, work_center, qty):  
        t = 0
        dist, mean, std = self.t_process[(prod, work_center)]
        if dist == 1:
            t = np.sum([abs(np.random.normal(loc=mean, scale=std, size=qty))])      # distribución = 1 (Normal)  
        if dist == 2:
            t = np.sum([abs(np.random.uniform(low=mean, high=std, size=qty))])      # distribución = 2 (Uniforme)
        return t
        
    # devuelve el código y la cantidad de materia prima 
    # que consume por producto teniendo en cuenta la 
    # cantidad y centro de trabajo.    

    def get_rawqty(self, prod, work_center, qty, color):
        raw_code = 'madera'
        if work_center=='corte_m': raw_code = 'metal'
        if work_center=='pintura': raw_code = color
        q = self.t_rawqty[(prod, work_center)]*qty
        return raw_code, q

    # transforma minutos de simulación en fechas 
    # reales, teniendo en cuenta dia de la semana, 
    # horario y feriados.

    def get_date(self, env, tiempo=-1):
        if tiempo==-1: tiempo = self.env.now                                               
        dias = int(tiempo/self.worked_minutes)
        day = self.start_date
        day = day + dt.timedelta(days=1)
        while dias>0:    
            if (day.weekday()<5) and ((day.day, day.month, day.year) not in self.holidays): dias -= 1 
            day = day + dt.timedelta(days=1)
        day = day + dt.timedelta(days=-1)
        dif = ((tiempo/self.worked_minutes)-int(tiempo/self.worked_minutes))*self.worked_minutes/60
        if dif>3: dif += 1
        day = day + dt.timedelta(hours=9+dif)
        return day

    # --------------------------------------------------------------------------------------------------------

    # OPERACION ESTANDARIZADA

    def operation(self, env, work_center, product_code, lot, qty, next_wc, color):
    
        # MANTENIMIENTO MAQUINA DE DOBLADO
        if (work_center == 'doblado') and (self.mant == True):                          # si doblado esta en mantenimiento se queda en un loop de espera
            while self.mant:   yield self.env.timeout(1)

        # MATERIA PRIMA
        if work_center in ('corte_m', 'corte_w', 'pintura'):                            # si es una operación que consume materia prima ... 
            raw_code, raw_q = self.get_rawqty(product_code, work_center, qty, color)    # obtiene el código de materia prima y la cantidad
            req_mp = self.raw_material[raw_code].get(qty*raw_q)                         # prepara el pedido
            yield req_mp                                                                # espera hasta que el pedido este disponible
                
        # REQUERIMIENTO DE MANO DE OBRA                                                 
        if work_center!='corte_m':                                                      # si no es corte_m entonces hace el pedido de mano de obra ...
            worker_req = self.workers[work_center].request()                            
            yield worker_req                                                            # espera hasta obtenerlo
        else:                                                                           # corte_m es un contenedor y trabaja diferente
            while self.workers_corte_m.level==0:                                        # mientras no haya operarios ...
                jj = ceil(self.env.now/self.worked_minutes)                             # redondea para arriba ...   
                yield self.env.timeout( (jj*self.worked_minutes) - self.env.now + 1 )   # espera hasta el siguiente día
            yield self.workers_corte_m.get(1)                                           # pide un operario
        
        # SETUP (PINTURA)   
        if work_center=='pintura':                                                      # si es pintura ...
            yield self.env.timeout(self.get_setupTime(color))                           # genera una demora para el setup
        
        # PROCESO DE PRODUCTO
        process_time = self.get_processTime(product_code, work_center, qty)             # obtiene el tiempo de proceso y ...
        yield self.env.timeout(process_time)                                            # genera una demora


        # LIBERA MANO DE OBRA
        if work_center!='corte_m':                                                      
            yield self.workers[work_center].release(worker_req)                         # si no es corte_m libera el recurso con release
        else:
            yield self.workers_corte_m.put(1)                                           # si es corte_m devuelve una unidad al contenedor

        # AGREGA EL SEMIELABORADO/TERMINADO EN EL ALMACEN CORRESPONDIENTE               # OPERACION IMPORTANTE
        yield self.store[next_wc].put([lot, product_code])                              # pone el semielaborado en el almacen de la siguiente operación

        # CONTROL DE FINALIZACIÓN                                                       # finaliza si se realizó la última operación o se alcanzó el tiempo máximo
        if ((next_wc=='terminado') and (len(self.store['terminado'].items)>=len(self.seq['despacho']))) or (self.env.now>self.tmax):
            self.continuar = False
            self.end_date = self.get_date(self.env)
            self.end_min = self.env.now
    
    # --------------------------------------------------------------------------------------------------------

    # FUNCIONES PRINCIPALES
    
    # proceso encargado de asignar los trabajos 
    # siguiendo la secuencia planificada para un 
    # determinado work center.

    def main_operation(self, env, work_center):                                        
        idx = 0                                                                                     # índice de orden
        while self.continuar:
            i1 = yield self.store[work_center].get(lambda i: i[0]==self.seq[work_center][idx][0])   # espera a que el semielaborado del work center esté disponible en el almacén
            prod = self.seq[work_center][idx][1]                                                    # obtiene la información de producción
            lot = self.seq[work_center][idx][0]
            qty = self.seq[work_center][idx][2]
            next_wc = self.seq[work_center][idx][3]
            color = self.seq[work_center][idx][4]
            if next_wc is None: next_wc = 'terminado'                                               # considera el almacén final de productos terminados
            env.process(self.operation(env, work_center, prod, lot, qty, next_wc, color))           # genera el proceso que realiza la tarea
            idx += 1                                                                                # aumenta el orden
            if (idx>=len(self.seq[work_center])): break                                             # si se llega al máximo, entonces termina

    # proceso que controla el ausentismo 
    # acorde a la probabidad de ausentismo 
    # calculada.

    def ausentismo(self, env):                  
        while self.continuar:                                                                       # mientras la simulación no haya finalizado ...
            ausentes = sum(np.random.choice(a=[1,0],                                                # calcula la probabilidad de ausentismo de los dos operarios
                                            p=[self.p_ausentismo, 1-self.p_ausentismo], 
                                            size=self.workers_corte_m.capacity))
            if ausentes>0:                                                                          # si hay ausentes ...
                yield self.workers_corte_m.get(ausentes)                                            # los retiene por un día
                yield self.env.timeout(self.worked_minutes-1)
                yield self.workers_corte_m.put(ausentes)                                            # finalmente los "libera"

            # espera hasta el día siguiente ...
            yield self.env.timeout((ceil(self.env.now/self.worked_minutes)*self.worked_minutes) - self.env.now)
                
    # proceso que monitorea el 
    # reaprovisionamiento de materia prima 
    # acorde al calendario de entregas.

    def reaprovisionamiento(self, env):
        self.next_reaprov = self.reaprov[0]                                                         # la variable next_reaprov obtiene la fecha en minutos de la siguiente entrega      
        while self.continuar:
            self.reaprov.remove(self.next_reaprov)                                                  # elimina la entrega de la lista
            yield self.env.timeout(self.next_reaprov[0])                                            # espera hasta el tiempo de la entrega
            self.raw_material[self.next_reaprov[1]].put(self.next_reaprov[2])                       # ingresa la materia prima al almacén
            if len(self.reaprov) == 0: break                                                        # si no hay más entregas, termina 
            self.next_reaprov = self.reaprov[0]                                                     # la variable next_reaprov obtiene la fecha en minutos de la siguiente entrega      

    # proceso que monitorea la rotura de la 
    # dobladora en función de las distribuciones 
    # de TBF y TTR.
    
    def mantenimiento(self, env):
        # se harcodea intencionalmente la fecha de la primera rotura para que se vea el impacto en la simulación.
        self.next_fail = 1200 # abs(f.rvs(self.a_tbf, b_tbf, c_tbf, d_tbf)-self.last_maintenance) 
        while self.continuar:
            yield self.env.timeout(self.next_fail)                                                  # espera hasta la siguiente rotura
            self.ttr = abs(np.random.normal(self.u_ttr,self.d_ttr))                                 # calcula el tiempo de reparación
            self.mant = True                                                                        # configura el flag de mantenimiento
            yield self.env.timeout(self.ttr)                                                        # espera hasta la reparación
            self.next_fail = abs(f.rvs(self.a_tbf, b_tbf, c_tbf, d_tbf))                            # calcula la fecha de la siguiente rotura
            self.mant = False                                                                       # configura el flag de mantenimiento
            if self.next_fail>self.tmax: break                                                      # si se llego al tiempo máximo de simulación finaliza el proceso

    # --------------------------------------------------------------------------------------------------------

    # proceso que ejecuta la simulación ...

    def run(self):

        self.env.process(self.ausentismo(self.env))             # monitorea el ausentismo

        for i in list(self.wc.keys())[:-1]:
            self.env.process(self.main_operation(self.env, i))  # genera todas las funciones de control de tareas para cada centro de trabajo

        self.env.process(self.reaprovisionamiento(self.env))    # monitorea el reaprovisionamiento

        self.env.process(self.mantenimiento(self.env))          # monitorea la rotura y mantenimiento de las maquinas

        self.env.run()                                          # ejecuta la simulación
        
        return self.end_min

## EJECUCIÓN DE LA SIMULACIÓN

In [4]:
s = modelo1(start_date = start_date, 
            start_seq = start_seq,
            seq = seq, 
            wc = wc, 
            t_process = t_process, 
            t_rawqty = t_rawqty, 
            color_matrix = setup_color_matrix, 
            p_ausentismo = .15 , #p_ausentismo, 
            holidays = holidays, 
            stk = stk, 
            reaprov = reaprovisionamiento, 
            u_ttr = u_ttr, 
            d_ttr = d_ttr,
            a_tbf = a_tbf,
            b_tbf = b_tbf,
            c_tbf = c_tbf,
            d_tbf = d_tbf,
            tmax = 20000)

end_min = s.run()

print(f'FECHA FINALIZACIÓN ÚLTIMO LOTE: {s.end_date.strftime("%d-%m-%Y, %H:%M:%S")} ({s.end_min:.2f} minutos).\n')

for i in stk:
    s1 = f'STOCK ALMACEN {i.upper()}: '
    s2 = f'{s.raw_material[i].level:.2f} UND.'
    print(f'{s1.ljust(30)} {s2.rjust(15)}')


FECHA FINALIZACIÓN ÚLTIMO LOTE: 31-10-2024, 16:17:16 (9977.28 minutos).

STOCK ALMACEN MADERA:               67.80 UND.
STOCK ALMACEN METAL:                70.70 UND.
STOCK ALMACEN LACA:                 65.00 UND.
STOCK ALMACEN BLANCO:               21.30 UND.
STOCK ALMACEN CREMA:               110.00 UND.
STOCK ALMACEN MARRON:               77.00 UND.
STOCK ALMACEN DORADO:               42.00 UND.
STOCK ALMACEN ROJO:                  9.60 UND.
STOCK ALMACEN ALUMINIO:            202.00 UND.
STOCK ALMACEN NEGRO:                20.00 UND.
